# Fine-tuning a pretrained model
<https://huggingface.co/docs/transformers/v4.14.1/en/training>


In [1]:

from datasets import load_dataset
raw_datasets = load_dataset("imdb")

Found cached dataset imdb (C:/Users/kinha/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

The raw_datasets object is a dictionary with three keys: "train", "test" and "unsupervised" (which correspond to the three splits of that dataset). We will use the "train" split for training and the "test" split for validation.

To preprocess our data, we will need a tokenizer:

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


Parameter 'function'=<function tokenize_function at 0x00000264C4B3A290> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

You can learn more about the map method or the other ways to preprocess the data in the 🤗 Datasets documentation.

Next we will generate a small subset of the training and validation set, to enable faster training:

In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"],

In all the examples below, we will always use small_train_dataset and small_eval_dataset. Just replace them by their full equivalent to train or evaluate on the full dataset.

Since PyTorch does not provide a training loop, the 🤗 Transformers library provides a Trainer API that is optimized for 🤗 Transformers models, with a wide range of training options and with built-in features like logging, gradient accumulation, and mixed precision.

First, let’s define our model:

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2),

loading configuration file config.json from cache at C:\Users\kinha/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cache at C:\Users\kinha/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846

This will issue a warning about some of the pretrained weights not being used and some weights being randomly initialized. That’s because we are throwing away the pretraining head of the BERT model to replace it with a classification head which is randomly initialized. We will fine-tune this model on our task, transferring the knowledge of the pretrained model to it (which is why doing this is called transfer learning).

Then, to define our Trainer, we will need to instantiate a TrainingArguments. This class contains all the hyperparameters we can tune for the Trainer or the flags to activate the different training options it supports. Let’s begin by using all the defaults, the only thing we then have to provide is a directory in which the checkpoints will be saved:

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Then we can instantiate a Trainer like this:

In [13]:

from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)


AttributeError: 'tuple' object has no attribute 'to'

To fine-tune our model, we just need to call

In [8]:
trainer.train()

NameError: name 'trainer' is not defined

which will start a training that you can follow with a progress bar, which should take a couple of minutes to complete (as long as you have access to a GPU). It won’t actually tell you anything useful about how well (or badly) your model is performing however as by default, there is no evaluation during training, and we didn’t tell the Trainer to compute any metrics. Let’s have a look on how to do that now!

To have the Trainer compute and report metrics, we need to give it a compute_metrics function that takes predictions and labels (grouped in a namedtuple called EvalPrediction) and return a dictionary with string items (the metric names) and float values (the metric values).

The 🤗 Datasets library provides an easy way to get the common metrics used in NLP with the load_metric function. here we simply use accuracy. Then we define the compute_metrics function that just convert logits to predictions (remember that all 🤗 Transformers models return the logits) and feed them to compute method of this metric.

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)